# DDoS Classifier

## Notebook for Training and Testing DDoS detection

In [1]:
import os
import pandas as pd
import datetime
import calendar
import numpy as np
from sklearn.preprocessing import *
import pickle
import time

In [2]:
import tensorflow as tf
import keras as kr

/home/talhajavaid32/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
from keras import backend as K
from keras.utils import np_utils
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras.regularizers import l2

### New Experiment function

In [4]:
def create_new_experiment_dir(exp_base_dir):
    def get_unique_name(): return time.strftime("%Y%m%d-%H%M%S")

    exp_base_dir = os.path.abspath(os.path.expanduser(exp_base_dir))
    if exp_base_dir[-1] != '/':
        exp_base_dir += '/'

    # Create new experiment directory
    exp_dir = exp_base_dir + get_unique_name() + '/'
    os.makedirs(exp_dir)

    os.makedirs(exp_dir + 'pickle_dump/')
    os.makedirs(exp_dir + 'logs/')

    return exp_dir

### F1 Score

In [5]:
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall))

### Confusion Matrix

In [6]:
def print_confusion_matrix(plabels,tlabels):
    """
        functions print the confusion matrix for the different classes
        to find the error...
        
        Input:
        -----------
        plabels: predicted labels for the classes...
        tlabels: true labels for the classes
        
        code from: http://stackoverflow.com/questions/2148543/how-to-write-a-confusion-matrix-in-python
    """
    import pandas as pd
    plabels = pd.Series(np.squeeze(plabels))
    tlabels = pd.Series(np.squeeze(tlabels))
    
    # draw a cross tabulation...
    df_confusion = pd.crosstab(tlabels,plabels, rownames=['Actual'], colnames=['Predicted'], margins=True)
    
    #print df_confusion
    return df_confusion

# Cleaning Data

### Loading converted IDS 2017 Benign data

In [7]:
ids_benign_data = pd.read_csv(
    './data/IDS2017Data/Benign-MondayData/intraflow_data/Monday-WorkingHours.csv')
ids_benign_data

key             sa              da  \
0             825425012619216810580491880  8.254.250.126    192.168.10.5   
1            8253185121192168101480494860  8.253.185.121   192.168.10.14   
2                    19216810922400220064   192.168.10.9      224.0.0.22   
3           19216810922400252603725355504   192.168.10.9     224.0.0.252   
4            1921681091921681031371373292   192.168.10.9    192.168.10.3   
5            1921681091921681036368353424   192.168.10.9    192.168.10.3   
6            1921681091921681036327653424   192.168.10.9    192.168.10.3   
7            1921681091921681035636853160   192.168.10.9    192.168.10.3   
8            1921681091921681036288253160   192.168.10.9    192.168.10.3   
9           19216810919216810362884389654   192.168.10.9    192.168.10.3   
10          19216810919216810362885389740   192.168.10.9    192.168.10.3   
11          19216810919216810310281352768   192.168.10.9    192.168.10.3   
12        1921681091921681031029496716350   192.168.10.9    192.168.10.3   
13          19216810919216810310304459854   192.168.10.9    192.168.10.3   
14            192168109192168103103188880   192.168.10.9    192.168.10.3   
15           1921681091921681031032883690   192.168.10.9    192.168.10.3   
16           1921681091921681031033886140   192.168.10.9    192.168.10.3   
17           1921681091921681031034885614   192.168.10.9    192.168.10.3   
18           1921681091921681036543953384   192.168.10.9    192.168.10.3   
19          19216810919216810365440389654   192.168.10.9    192.168.10.3   
20          19216810919216810310353899818   192.168.10.9    192.168.10.3   
21           1921681091921681031036886178   192.168.10.9    192.168.10.3   
22           1921681031921681016199553163   192.168.10.3    192.168.10.1   
23          19216810322400252511285355756   192.168.10.3     224.0.0.252   
24         192168103224002525938853551620   192.168.10.3     224.0.0.252   
25         192168103224002525376153551620   192.168.10.3     224.0.0.252   
26            192168103192168101137137150   192.168.10.3    192.168.10.1   
27         192168103224002525669753551620   192.168.10.3     224.0.0.252   
28         192168103224002525014653551620   192.168.10.3     224.0.0.252   
29          19216810322400252506725355774   192.168.10.3     224.0.0.252   
...                                   ...            ...             ...   
486645      19216810145284143142600064431  192.168.10.14   52.84.143.142   
486646      19216810513516010086533444430  192.168.10.51   35.160.100.86   
486647      19216810191921681035259853422  192.168.10.19    192.168.10.3   
486648      19216810191921681034157753294  192.168.10.19    192.168.10.3   
486649       1921681031921681016016853233   192.168.10.3    192.168.10.1   
486650       1921681031921681016224553169   192.168.10.3    192.168.10.1   
486651      19216810145284145252599974431  192.168.10.14   52.84.145.252   
486652      19216810145284145252599964431  192.168.10.14   52.84.145.252   
486653      19216810191921681033680053208  192.168.10.19    192.168.10.3   
486654      19216810191921681031058853300  192.168.10.19    192.168.10.3   
486655   19216810191929916159557964433278  192.168.10.19   192.99.16.159   
486656       1921681019192168103860253208  192.168.10.19    192.168.10.3   
486657      19216810191921681032114353208  192.168.10.19    192.168.10.3   
486658       1921681019192168103723253300  192.168.10.19    192.168.10.3   
486659      19216810191921681034470453208  192.168.10.19    192.168.10.3   
486660  192168101915869126206487864432352  192.168.10.19  158.69.126.206   
486661  192168101915869126206487874432400  192.168.10.19  158.69.126.206   
486662    1921681019192991615955799443667  192.168.10.19   192.99.16.159   
486663   19216810191586912620648789443754  192.168.10.19  158.69.126.206   
486664     192168101418424107198600104431  192.168.10.14  184.24.107.198   
486665     192168101617221762065869444392  192.168.10.16   172.217.6.206   
486666    

### Loading converted IDS 2017 DoS/DDoS data

In [8]:
ids_ddos_data = pd.read_csv(
    './data/IDS2017Data/DDoS-WednesdayData/intraflow_data/Wednesday-workingHours.csv')
ids_ddos_data

key             sa               da  \
0       19216810152392552552504979219001507  192.168.10.15  239.255.255.250   
1        4083143209192168101444349461132340  40.83.143.209    192.168.10.14   
2           1921681017224002515353535325487  192.168.10.17      224.0.0.251   
3            192168101922400251535353531485  192.168.10.19      224.0.0.251   
4        1921681014408314320949460443131550  192.168.10.14    40.83.143.209   
5                     192168101722400220016  192.168.10.17       224.0.0.22   
6             19216810171921681033449253242  192.168.10.17     192.168.10.3   
7            192168101615869223100123123192  192.168.10.16   158.69.223.100   
8               192168101615920387212312396  192.168.10.16     159.203.8.72   
9             19216810171921681031522153242  192.168.10.17     192.168.10.3   
10            19216810193812611310123123576  192.168.10.19    38.126.113.10   
11              192168101999224253912312396  192.168.10.19     99.224.25.39   
12            19216810191321634101123123480  192.168.10.19    132.163.4.101   
13         19216810171921681025513713715510  192.168.10.17   192.168.10.255   
14          1921681017192168102551381382585  192.168.10.17   192.168.10.255   
15            19216810501921681017137137124  192.168.10.50    192.168.10.17   
16             1921681017192168103123123960  192.168.10.17     192.168.10.3   
17             1921681017192168103318653200  192.168.10.17     192.168.10.3   
18            19216810171921681033174453258  192.168.10.17     192.168.10.3   
19         19216810121921681035965632682994  192.168.10.12     192.168.10.3   
20          1921681012192168103438323894148  192.168.10.12     192.168.10.3   
21             1921681016381261131012312396  192.168.10.16    38.126.113.10   
22           192168101910413153252123123576  192.168.10.19   104.131.53.252   
23           192168101915869223100123123384  192.168.10.19   158.69.223.100   
24          1921681019128100100128123123480  192.168.10.19  128.100.100.128   
25               19216810194531607512312396  192.168.10.19      4.53.160.75   
26              192168101920881124412312396  192.168.10.19     208.81.1.244   
27          1921681091921681031378444514394   192.168.10.9     192.168.10.3   
28            19216810171921681033605253200  192.168.10.17     192.168.10.3   
29            19216810171921681035615353258  192.168.10.17     192.168.10.3   
...                                     ...            ...              ...   
615220        19216810141921681034984553426  192.168.10.14     192.168.10.3   
615221     19216810144070221249586684435300  192.168.10.14    40.70.221.249   
615222      1921681050192168102551381384510  192.168.10.50   192.168.10.255   
615223      1921681012192168103388083895006  192.168.10.12     192.168.10.3   
615224     19216810121921681035464632688272  192.168.10.12     192.168.10.3   
615225        19216810121921681050370821390  192.168.10.12    192.168.10.50   
615226         1921681050192168103123453108  192.168.10.50     192.168.10.3   
615227         1921681050192168103263153108  192.168.10.50     192.168.10.3   
615228        19216810501921681036413453108  192.168.10.50     192.168.10.3   
615229        19216810501921681036451253108  192.168.10.50     192.168.10.3   
615230        19216810501921681032034553108  192.168.10.50     192.168.10.3   
615231        19216810501921681033100953108  192.168.10.50     192.168.10.3   
615232        19216810501921681036534553108  192.168.10.50     192.168.10.3   
615233        19216810501921681034068453108  192.168.10.50     192.168.10.3   
615234        19216810501921681032755853108  192.168.10.50     192.168.10.3   
615235        19216810501921681035604553108  192.168.10.50     192.168.10.3   
615236        19216810501921681034941353108  192.168.10.50     192.168.10.3   
615237        19216810501921681031537853108  192.168.10.50     192.168.10.3   
615238        19216810501921681031774153108  192.168.10.50     192.168.10.3   
615239     

### Loading converted App DoS data

In [9]:
app_dos_data = pd.read_csv('./data/SlowDosData/intraflow_data/AppDDos.csv')
app_dos_data

key              sa               da  \
0         192168110119216851224175223872   192.168.1.101    192.168.5.122   
1       12180551401921683115803816538828   12.180.55.140    192.168.3.115   
2        1921684121192168512251061226172   192.168.4.121    192.168.5.122   
3           8298861831921684119804491696    82.98.86.183    192.168.4.119   
4         911952401241921684119804490406  91.195.240.124    192.168.4.119   
5           1921684119213155642094489800   192.168.4.119   213.155.64.209   
6          19216841191921684255138138210   192.168.4.119    192.168.4.255   
7       19216811041921685122158372218208   192.168.1.104    192.168.5.122   
8          19216841191921685122297153372   192.168.4.119    192.168.5.122   
9           1921685122198164302504353223   192.168.5.122     198.164.30.2   
10         19216841216554811715106280763   192.168.4.121     65.54.81.171   
11           192168412165548111751060800   192.168.4.121     65.54.81.117   
12            19216821102122270823914800   192.168.2.110     212.227.0.82   
13         19216821102122271161113916800   192.168.2.110  212.227.116.111   
14         19216831151921683255138138201   192.168.3.115    192.168.3.255   
15         19216851221921681103143109034   192.168.5.122    192.168.1.103   
16         19216821102122271161713917800   192.168.2.110  212.227.116.171   
17          1921682110911952401243905800   192.168.2.110   91.195.240.124   
18           192168311514216614703703800   192.168.3.115    142.166.14.70   
19         19216851222240025153535353628   192.168.5.122      224.0.0.251   
20         19216851221921683114222672912   192.168.5.122    192.168.3.114   
21           192168512219216831140013488   192.168.5.122    192.168.3.114   
22           192168311514216614803704800   192.168.3.115    142.166.14.80   
23       1921683114192168512226732236680   192.168.3.114    192.168.5.122   
24        192168110119216851224176224604   192.168.1.101    192.168.5.122   
25           192168211019291247253913800   192.168.2.110    192.91.247.25   
26           192168110467189911515828800   192.168.1.104     67.18.99.115   
27         19216831151921685122103253917   192.168.3.115    192.168.5.122   
28         19216851221981643024530353430   192.168.5.122     198.164.30.2   
29      19216831151421661485381880300165   192.168.3.115    142.166.14.85   
...                                  ...             ...              ...   
270272     19216811011921685122228422160   192.168.1.101    192.168.5.122   
270273     19216821061921685122367722160   192.168.2.106    192.168.5.122   
270274      1921685122224002515353535388   192.168.5.122      224.0.0.251   
270275     19216821121921685122350722160   192.168.2.112    192.168.5.122   
270276     19216811011921685122228522160   192.168.1.101    192.168.5.122   
270277     19216821061921685122367822160   192.168.2.106    192.168.5.122   
270278     19216831141921685122223422160   192.168.3.114    192.168.5.122   
270279    192168211219216851223508223748   192.168.2.112    192.168.5.122   
270280     19216811011921685122228622160   192.168.1.101    192.168.5.122   
270281     19216821061921685122367922160   192.168.2.106    192.168.5.122   
270282        19216851221921682108002960   192.168.5.122    192.168.2.108   
270283     19216821081921685122151522160   192.168.2.108    192.168.5.122   
270284      1921683114192168325513713750   192.168.3.114    192.168.3.255   
270285     19216811011921685122228722160   192.168.1.101    192.168.5.122   
270286     19216821061921685122368022160   192.168.2.106    192.168.5.122   
270287    192168110119216851222288223748   192.168.1.101    192.168.5.122   
270288    192168210619216851223681223748   192.168.2.106    192.168.5.122   
270289     19216821081921685122151622160   192.168.2.108    192.168.5.122   
270290     19216851221921682111143111234   192.168.5.122    192.168.2.111   
270291    192168110519216851221924422160   192.168.1.105    192.168.5.122   
270292    19216821081921685122151

### Load Botnet data later...

### Merge Data

In [11]:
data_df = pd.concat((ids_benign_data, app_dos_data, ids_ddos_data), ignore_index=True)
data_df

key             sa              da  \
0             825425012619216810580491880  8.254.250.126    192.168.10.5   
1            8253185121192168101480494860  8.253.185.121   192.168.10.14   
2                    19216810922400220064   192.168.10.9      224.0.0.22   
3           19216810922400252603725355504   192.168.10.9     224.0.0.252   
4            1921681091921681031371373292   192.168.10.9    192.168.10.3   
5            1921681091921681036368353424   192.168.10.9    192.168.10.3   
6            1921681091921681036327653424   192.168.10.9    192.168.10.3   
7            1921681091921681035636853160   192.168.10.9    192.168.10.3   
8            1921681091921681036288253160   192.168.10.9    192.168.10.3   
9           19216810919216810362884389654   192.168.10.9    192.168.10.3   
10          19216810919216810362885389740   192.168.10.9    192.168.10.3   
11          19216810919216810310281352768   192.168.10.9    192.168.10.3   
12        1921681091921681031029496716350   192.168.10.9    192.168.10.3   
13          19216810919216810310304459854   192.168.10.9    192.168.10.3   
14            192168109192168103103188880   192.168.10.9    192.168.10.3   
15           1921681091921681031032883690   192.168.10.9    192.168.10.3   
16           1921681091921681031033886140   192.168.10.9    192.168.10.3   
17           1921681091921681031034885614   192.168.10.9    192.168.10.3   
18           1921681091921681036543953384   192.168.10.9    192.168.10.3   
19          19216810919216810365440389654   192.168.10.9    192.168.10.3   
20          19216810919216810310353899818   192.168.10.9    192.168.10.3   
21           1921681091921681031036886178   192.168.10.9    192.168.10.3   
22           1921681031921681016199553163   192.168.10.3    192.168.10.1   
23          19216810322400252511285355756   192.168.10.3     224.0.0.252   
24         192168103224002525938853551620   192.168.10.3     224.0.0.252   
25         192168103224002525376153551620   192.168.10.3     224.0.0.252   
26            192168103192168101137137150   192.168.10.3    192.168.10.1   
27         192168103224002525669753551620   192.168.10.3     224.0.0.252   
28         192168103224002525014653551620   192.168.10.3     224.0.0.252   
29          19216810322400252506725355774   192.168.10.3     224.0.0.252   
...                                   ...            ...             ...   
1372197     19216810141921681034984553426  192.168.10.14    192.168.10.3   
1372198  19216810144070221249586684435300  192.168.10.14   40.70.221.249   
1372199   1921681050192168102551381384510  192.168.10.50  192.168.10.255   
1372200   1921681012192168103388083895006  192.168.10.12    192.168.10.3   
1372201  19216810121921681035464632688272  192.168.10.12    192.168.10.3   
1372202     19216810121921681050370821390  192.168.10.12   192.168.10.50   
1372203      1921681050192168103123453108  192.168.10.50    192.168.10.3   
1372204      1921681050192168103263153108  192.168.10.50    192.168.10.3   
1372205     19216810501921681036413453108  192.168.10.50    192.168.10.3   
1372206     19216810501921681036451253108  192.168.10.50    192.168.10.3   
1372207     19216810501921681032034553108  192.168.10.50    192.168.10.3   
1372208     19216810501921681033100953108  192.168.10.50    192.168.10.3   
1372209     19216810501921681036534553108  192.168.10.50    192.168.10.3   
1372210     19216810501921681034068453108  192.168.10.50    192.168.10.3   
1372211     19216810501921681032755853108  192.168.10.50    192.168.10.3   
1372212     19216810501921681035604553108  192.168.10.50    192.168.10.3   
1372213     19216810501921681034941353108  192.168.10.50    192.168.10.3   
1372214     19216810501921681031537853108  192.168.10.50    192.168.10.3   
1372215     19216810501921681031774153108  192.168.10.50    192.168.10.3   
1372216     19216810501921681031787553108  192.168.10.50    192.168.10.3   
1372217     19216810501921681036274053108  192.168.10.50    192.168.10.3   
1372218   

In [12]:
data_df.shape

(1372227, 668)

### Imputing NaNs

In [21]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print data_df.isnull().sum()

MemoryError: 

## Old cells below

In [ ]:
dt2ts = lambda dt: float(calendar.timegm(dt.timetuple()))

In [ ]:
ftp_st_dt = datetime.datetime(2017, 7, 4, 9, 20)
ftp_et_dt = datetime.datetime(2017, 7, 4, 10, 20)

ftp_st = dt2ts(ftp_st_dt)
ftp_et = dt2ts(ftp_et_dt)

ssh_st_dt = datetime.datetime(2017, 7, 4, 14, 0)
ssh_et_dt = datetime.datetime(2017, 7, 4, 15, 0)

ssh_st = dt2ts(ssh_st_dt)
ssh_et = dt2ts(ssh_et_dt)

print ftp_st
print ftp_et
print ssh_st
print ssh_et

# NetFlow

In [ ]:
import os
import pandas as pd
import datetime
import calendar
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from category_encoders import BinaryEncoder

# Loading NetFlow Data

In [ ]:
path = './data/netflow_data/'

#### Load all days

In [ ]:
# Load all days
files = os.listdir(path)
data = {}
for f in files:
    data[f.replace('.csv', '')] =  pd.read_csv(path + f)

In [ ]:
# Filenames
data.keys()

#### Fix column headers

In [ ]:
for k, d in data.iteritems():
    d.columns = d.columns.str.lower().str.strip().str.replace(
        ' ', '_').str.replace('/s', '').str.replace('/', '_').str.replace('.', '_')

#### Features

In [ ]:
feat = set()
for k, v in data.iteritems():
    object_cols = list(v.dtypes[v.dtypes == 'object'].keys())
    all_cols = list(v.columns)
    feat.update(list(filter(lambda x: x not in object_cols, all_cols)))
# These features have type object so keeping them to convert to float64
feat.add('flow_bytes') 
feat.add('flow_packets')
feat = list(feat)

#### Merging Data

In [ ]:
# has NaNs deal later
days_to_ignore = ['Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX']

In [ ]:
cols = feat + ['label'] # Columns to keep from data
data_df = pd.DataFrame(columns=cols)
for k, d in data.iteritems():
    if k not in days_to_ignore:
        data_df = pd.concat((data_df, d[cols]), ignore_index=True)
data_df

#### Fix dtypes

In [ ]:
data_df[feat] = data_df[feat].astype(pd.np.float64)

# Preprocessing

#### Imputing NaNs

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print data_df.isnull().sum()

In [ ]:
data_df['flow_bytes'][data_df['flow_bytes'].isnull()] = 0

In [ ]:
print 'No NaNs' if sum(data_df.isnull().any()) == 0 else 'Found NaNs'

#### Removing Infs

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print (data_df == float('Inf')).sum()

In [ ]:
data_df = data_df.replace([np.inf, -np.inf], np.nan).dropna()

## Writing data

In [ ]:
data_df.to_csv('./data/netflow.csv', index=False)

## Loading processed

In [ ]:
import os
import pandas as pd
import datetime
import calendar
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from category_encoders import BinaryEncoder

In [ ]:
data_df = pd.read_csv('./data/netflow.csv')
feat = list(data_df.columns)
feat.remove('label')

#### Ignoring encoding for now

#### One hot encode categorical

In [ ]:
# categ_feat = ['source_port', 'destination_port', 'protocol']

# data_df = data.itervalues().next()
# data_df = pd.get_dummies(data_df, prefix=['sp', 'dp', 'pr'], columns=categ, sparse=True)
# data_df

In [ ]:
# enc = OneHotEncoder()
# enc.fit(data_df)
# transformed = enc.transform(data_df)

#### Scaling data

In [ ]:
from sklearn.preprocessing import *
import pickle

#### Dropping features having very close max and min values to avoid NaNs on scaling

In [ ]:
diff = data_df[feat].max(axis=0) - data_df[feat].min(axis=0)
eps = 0.00000000001

feat = list(data_df[feat].columns[~(diff.abs() < eps)])

In [ ]:
# Considering all features numerical for now  Note: source_port, destination_port, protocol are
num_feat = feat

# Using MinMaxScaler
min_max = MinMaxScaler()
# Fit scaler
min_max.fit(data_df[num_feat].as_matrix())
# Save scaler using pickle
pickle.dump(min_max, open("min_max_data.p", "wb"))
# Transform data
data_df[num_feat] = pd.DataFrame(min_max.transform(
    data_df[num_feat].as_matrix()), columns=num_feat)

#### Encoding labels

In [ ]:
le = LabelEncoder()
le.fit(data_df['label'].as_matrix())
pickle.dump(le, open("label_encoder.p", "wb"))

data_df['label'] = pd.DataFrame(le.transform(data_df['label'].as_matrix()), columns=['label'])

#### Splitting up data

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = data_df[feat].as_matrix()
y = data_df['label'].as_matrix()

In [ ]:
# Splitting Train Test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.8, stratify=y)

In [ ]:
# Splitting Train Validation
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, train_size=0.8, stratify=y_train)

In [ ]:
print (X_train.shape)
print (y_train.shape)
print (X_val.shape)
print (y_val.shape)
print (X_test.shape)
print (y_test.shape)

In [ ]:
labels = dict(zip(list(le.classes_), le.transform(list(le.classes_))))
labels

In [ ]:
print 'Benign flows in Training data:', sum(y_train == 1)
print 'Malicious flows in Training data:', sum(y_train == 0)
print 'Benign flows in Validation data:', sum(y_val == 1)
print 'Malicious flows in Validation data:', sum(y_val == 0)

for k, v in labels.iteritems():
    print 
    

# Building the Model

In [ ]:
import tensorflow as tf
import keras as kr

In [ ]:
from keras import backend as K
from keras.utils import np_utils
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.layers import Dense, Activation, Flatten
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam, RMSprop

## Building the model

In [ ]:
# Training parameters
batch_size = 256 # 64 128 256
epochs = 600
feat = X_train.shape[1]
alpha = 0.01

In [ ]:
# Keras sequential model
model = Sequential()
model.add(Dense(1024, input_shape=(feat, ), activation='relu', kernel_initializer='he_normal'))
model.add(Dense(512, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(256, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(128, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(64, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(32, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(16, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(8, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(1, activation='sigmoid', kernel_initializer='VarianceScaling'))
adam = Adam(lr=alpha, decay=0.0005)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model.summary()

In [ ]:
%%time
checkpoint = ModelCheckpoint(filepath='./saved_model/24-Dec-Model-1.hdf5', verbose=1, save_best_only=True)
tensorboard = TensorBoard(log_dir='./logs', batch_size=batch_size)
res = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val), callbacks=[checkpoint, tensorboard]) 